In [1]:
import pandas as pd
import numpy as np

def make_tuple(cor):
    temp=cor.replace('(','').replace(')','')
    a=tuple([int(i) for i in temp.split(',')])
    return a

third_stage = pd.read_csv(r'\1Hz.csv')

total_corpus = []
ids = np.unique(third_stage['id'].to_numpy())
for ID in ids:
    print(ID)
    single_data = third_stage[third_stage['id']==ID].reset_index().drop(columns = ['index'])
    corpus = list(single_data['corpus'].to_numpy())
    for i in range(len(corpus)):
        corpus[i] = make_tuple(corpus[i])
        
    total_corpus.append(corpus)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [2]:
from gensim.test.utils import common_corpus
from gensim.models.ldamodel import LdaModel
from gensim.test.utils import datapath

ldamodel = LdaModel(total_corpus,
            chunksize=2000,
            alpha=1.50,
            eta=1.0,
            decay=0.8,
            iterations=1000,
            num_topics=8,
            passes=20,
            eval_every=1)

temp_file = datapath(r"\model.model")
ldamodel.save(temp_file)


In [ ]:
#test the effectiveness of ldamodel

test_cor = [words_to_cor(683),words_to_cor(528),words_to_cor(501),words_to_cor(714),words_to_cor(497),words_to_cor(714),words_to_cor(683)]
print(ldamodel[test_cor])
test_cor = [words_to_cor(657),words_to_cor(831),words_to_cor(840),words_to_cor(863),words_to_cor(687),words_to_cor(869),words_to_cor(807)]
print(ldamodel[test_cor])
test_cor = [words_to_cor(499),words_to_cor(683),words_to_cor(714),words_to_cor(684),words_to_cor(558),words_to_cor(528),words_to_cor(528)]
print(ldamodel[test_cor])
test_cor = [words_to_cor(373),words_to_cor(404),words_to_cor(404),words_to_cor(373),words_to_cor(404),words_to_cor(373),words_to_cor(372)]
print(ldamodel[test_cor])

In [5]:
import math

def array_to_cor(inputs_speed):
    array_speed = inputs_speed.cpu().numpy()
    cor_speed = []
    for i in range(6):
        cor_speed.append((int(array_speed[0,2*i+0]),int(array_speed[0,2*i+1])))
    return cor_speed

def words_to_cor(words):
    a = math.ceil(words/30)
    b = words % 30
    return (a,b)

def cor_to_words(cor):
    a,b = cor
    words = (a-1)*30.0+b
    return words

def lon_speed(lon):
    if lon != 6:
        return 2+(lon-1)*4
    else:
        return 22

def lat_speed(lat):
    if lat == 1:
        return -2.0
    elif lat == 5:
        return 2.0
    else:
        return -1.0+(lat-2)
    
def cor_to_speed(cor):
    a,b = cor
    a_lon = math.ceil(a/5)
    a_lat = a - (a_lon-1)*5
    b_lon = math.ceil(b/5)
    b_lat = b - (b_lon-1)*5
    
    return [a_lon,a_lat,b_lon,b_lat],[lon_speed(a_lon),lon_speed(b_lon)],[lat_speed(a_lat),lat_speed(b_lat)]

def batch_speed_trans(outputs):
    #outputs = outputs.item()
    lons = np.zeros([0,2])
    lats = np.zeros([0,2])
    for output in outputs:
        _,lon_v_pre,lat_v_pre = cor_to_speed(words_to_cor(output.item()))
        lons = np.concatenate([lons,np.array(lon_v_pre).reshape(1,-1)],axis=0)
        lats = np.concatenate([lats,np.array(lat_v_pre).reshape(1,-1)],axis=0)
    return lons,lats

In [9]:
def search_known(prediction):
    known_topics = []
    knowm_fea = []
    for i in range(len(prediction)):
        known_topics.append(prediction[i][0])
        knowm_fea.append(prediction[i][1])
    return known_topics,knowm_fea

def construct_feature(prediction,num_topic):
    known_topics,knowm_fea = search_known(prediction)
    #print(known_topics)
    temp_feature = []
    if len(known_topics)==num_topic:
        return knowm_fea
    for i in range(num_topic):
        completion = (1-sum(knowm_fea))/(num_topic-len(known_topics))
        if i in known_topics:
            indexs = known_topics.index(i)
            temp_feature.append(knowm_fea[indexs])
        else:
            temp_feature.append(completion)
    return temp_feature

In [10]:
def make_tuple(cor):
    temp=cor.replace('(','').replace(')','')
    a=tuple([int(i) for i in temp.split(',')])
    return a

ids = np.unique(third_stage['id'].to_numpy())
total_times = 0
num_topic = 8
train_data = []
single_length = []
for ID in ids:
    print(ID)
    single_data = third_stage[third_stage['id']==ID].reset_index().drop(columns = ['index'])

    for i in range(len(single_data)-6):
        #for j in range(i,i+7):
        corpus = [single_data.loc[i]['corpus'],single_data.loc[i+1]['corpus'],single_data.loc[i+2]['corpus'],
                      single_data.loc[i+3]['corpus'],single_data.loc[i+4]['corpus'],single_data.loc[i+5]['corpus'],
                      single_data.loc[i+6]['corpus']]
        tuple_corpus = []
        for cor in corpus:
            a=make_tuple(cor)
            tuple_corpus.append(a)
        temp_feature = construct_feature(ldamodel[tuple_corpus],num_topic)
        ID = single_data.loc[i+6]['id']
        t = single_data.loc[i+6]['t']
        corpus = make_tuple(single_data.loc[i+6]['corpus'])
        train_data.append([ID,t,len(single_data)-6,corpus,temp_feature,int(single_data.loc[i+6]['words'])])
    total_times += len(single_data)-6
    single_length.append(len(single_data)-6)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [11]:
def pad_seq(temp_train):
    np_train = np.array(temp_train)
    if np_train.shape[0]>15:
        print('error')
        return np_train
    elif np_train.shape[0]==15:
        return np_train
    else:
        loss_len = 15-np_train.shape[0]
        loss_train = np.zeros([loss_len,8])
        return np.concatenate([loss_train,np_train],axis=0)
    
ids = np.unique(third_stage['id'].to_numpy())
flag = 0
minimum = 4
maximum = 14
all_train = []
train_speed = []
target_speed = []
np_all_train = np.zeros([0,15,8])
#np_all_x_train = np.zeros([0,14,12])
#np_all_y_train = np.zeros([0,1,12])

for ID in ids:
    print(ID)
    single_data = third_stage[third_stage['id']==ID].reset_index().drop(columns = ['index'])
    length = single_length[int(ID)-1]
    temp_data = train_data[flag:flag+length]
    flag += length
    if length<4+1:
        continue
    for i in range(length-minimum+1):
        end_num = min(length-1,i+maximum)
        for j in range(i+minimum,end_num+1):
            next_speed = temp_data[j][5]
            current_speed = temp_data[j-1][5]
            train_speed.append(next_speed)
            target_speed.append(current_speed)
            temp_train = []
            for k in range(i,j+1):
                temp_train.append(temp_data[k][4])
            np_temp_train = pad_seq(temp_train).reshape([-1,pad_seq(temp_train).shape[0],pad_seq(temp_train).shape[1]])
            all_train.append(temp_train)
            np_all_train = np.concatenate([np_all_train,np_temp_train],axis=0)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [12]:
np_train_speed = np.array(train_speed).reshape(-1,1)
np_target_speed = np.array(target_speed).reshape(-1,1)
print(np_all_train.shape,np_train_speed.shape,np_target_speed.shape)

np.save(r'\np_all_train.npy',np_all_train)
np.save(r'\np_train_speed.npy',np_train_speed)
np.save(r'\np_target_speed.npy',np_target_speed)

(191699, 15, 8) (191699, 1) (191699, 1)


In [77]:
#10s data preprocess

def make_tuple(cor):
    temp=cor.replace('(','').replace(')','')
    a=tuple([int(i) for i in temp.split(',')])
    return a

ids = np.unique(third_stage['id'].to_numpy())
total_times = 0
num_topic = 8
train_data = []
single_length = []
for ID in ids:
    print(ID)
    single_data = third_stage[third_stage['id']==ID].reset_index().drop(columns = ['index'])

    for i in range(len(single_data)-6):
        #for j in range(i,i+7):
        corpus = [single_data.loc[i]['corpus'],single_data.loc[i+1]['corpus'],single_data.loc[i+2]['corpus'],
                      single_data.loc[i+3]['corpus'],single_data.loc[i+4]['corpus'],single_data.loc[i+5]['corpus'],
                      single_data.loc[i+6]['corpus']]
        tuple_corpus = []
        for cor in corpus:
            a=make_tuple(cor)
            tuple_corpus.append(a)
        temp_feature = construct_feature(ldamodel[tuple_corpus],num_topic)
        ID = single_data.loc[i+6]['id']
        t = single_data.loc[i+6]['t']
        corpus = make_tuple(single_data.loc[i+6]['corpus'])
        his_corpus = [make_tuple(single_data.loc[i+1]['corpus']),make_tuple(single_data.loc[i+2]['corpus']),
                     make_tuple(single_data.loc[i+3]['corpus']),make_tuple(single_data.loc[i+4]['corpus']),
                     make_tuple(single_data.loc[i+5]['corpus']),make_tuple(single_data.loc[i+6]['corpus'])]
        train_data.append([ID,t,len(single_data)-6,corpus,temp_feature,int(single_data.loc[i+6]['words']),his_corpus])
    total_times += len(single_data)-6
    single_length.append(len(single_data)-6)

1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
9.0
10.0
11.0
12.0
13.0
14.0
15.0
16.0
17.0
18.0
19.0
20.0
21.0
22.0
23.0
24.0
25.0
26.0
27.0
28.0
29.0
30.0
31.0
32.0
33.0
34.0
35.0
36.0
37.0
38.0
39.0
40.0
41.0
42.0
43.0
44.0
45.0
46.0
47.0
48.0
49.0
50.0
51.0
52.0
53.0
54.0
55.0
56.0
57.0
58.0
59.0
60.0
61.0
62.0
63.0
64.0
65.0
66.0
67.0
68.0
69.0
70.0
71.0
72.0
73.0
74.0
75.0
76.0
77.0
78.0
79.0
80.0
81.0
82.0
83.0
84.0
85.0
86.0
87.0
88.0
89.0
90.0
91.0
92.0
93.0
94.0
95.0
96.0
97.0
98.0
99.0
100.0
101.0
102.0
103.0
104.0
105.0
106.0
107.0
108.0
109.0
110.0
111.0
112.0
113.0
114.0
115.0
116.0
117.0
118.0
119.0
120.0
121.0
122.0
123.0
124.0
125.0
126.0
127.0
128.0
129.0
130.0
131.0
132.0
133.0
134.0
135.0
136.0
137.0
138.0
139.0
140.0
141.0
142.0
143.0
144.0
145.0
146.0
147.0
148.0
149.0
150.0
151.0
152.0
153.0
154.0
155.0
156.0
157.0
158.0
159.0
160.0
161.0
162.0
163.0
164.0
165.0
166.0
167.0
168.0
169.0
170.0
171.0
172.0
173.0
174.0
175.0
176.0
177.0
178.0
179.0
180.0
181.0
182.0
183.0
184.0
185.

In [78]:
#Process the speed data for the next 10 steps
def pad_seq(temp_train):
    np_train = np.array(temp_train)
    if np_train.shape[0]>14:
        print('error')
        return np_train
    elif np_train.shape[0]==14:
        return np_train
    else:
        loss_len = 14-np_train.shape[0]
        loss_train = np.zeros([loss_len,8])
        return np.concatenate([loss_train,np_train],axis=0)
    
ids = np.unique(third_stage['id'].to_numpy())
flag = 0
minimum = 4
maximum = 14
all_train = []
train_speed = []
train_topic = []
history_speed = []
np_all_train = np.zeros([0,14,8])

for ID in ids:
    print(ID)
    single_data = third_stage[third_stage['id']==ID].reset_index().drop(columns = ['index'])
    length = single_length[int(ID)-1]
    temp_data = train_data[flag:flag+length]
    flag += length
    if length<4+10:
        continue
    for i in range(length-minimum+1-10):
        end_num = min(length-10,i+maximum)
        for j in range(i+minimum-1,end_num):
            next_speed = [temp_data[j+1][5],temp_data[j+2][5],temp_data[j+3][5],temp_data[j+4][5],temp_data[j+5][5],temp_data[j+6][5],temp_data[j+7][5],temp_data[j+8][5],temp_data[j+9][5],temp_data[j+10][5]]
            next_topic = [temp_data[j+1][4],temp_data[j+2][4],temp_data[j+3][4],temp_data[j+4][4],temp_data[j+5][4],temp_data[j+6][4],temp_data[j+7][4],temp_data[j+8][4],temp_data[j+9][4],temp_data[j+10][4]]
            train_speed.append(next_speed)
            train_topic.append(next_topic)
            history_speed.append(temp_data[j][6])
            temp_train = []
            for k in range(i,j+1):
                temp_train.append(temp_data[k][4])
            np_temp_train = pad_seq(temp_train).reshape([-1,pad_seq(temp_train).shape[0],pad_seq(temp_train).shape[1]])
            all_train.append(temp_train)
            np_all_train = np.concatenate([np_all_train,np_temp_train],axis=0)


1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
9.0
10.0
11.0
12.0
13.0
14.0
15.0
16.0
17.0
18.0
19.0
20.0
21.0
22.0
23.0
24.0
25.0
26.0
27.0
28.0
29.0
30.0
31.0
32.0
33.0
34.0
35.0
36.0
37.0
38.0
39.0
40.0
41.0
42.0
43.0
44.0
45.0
46.0
47.0
48.0
49.0
50.0
51.0
52.0
53.0
54.0
55.0
56.0
57.0
58.0
59.0
60.0
61.0
62.0
63.0
64.0
65.0
66.0
67.0
68.0
69.0
70.0
71.0
72.0
73.0
74.0
75.0
76.0
77.0
78.0
79.0
80.0
81.0
82.0
83.0
84.0
85.0
86.0
87.0
88.0
89.0
90.0
91.0
92.0
93.0
94.0
95.0
96.0
97.0
98.0
99.0
100.0
101.0
102.0
103.0
104.0
105.0
106.0
107.0
108.0
109.0
110.0
111.0
112.0
113.0
114.0
115.0
116.0
117.0
118.0
119.0
120.0
121.0
122.0
123.0
124.0
125.0
126.0
127.0
128.0
129.0
130.0
131.0
132.0
133.0
134.0
135.0
136.0
137.0
138.0
139.0
140.0
141.0
142.0
143.0
144.0
145.0
146.0
147.0
148.0
149.0
150.0
151.0
152.0
153.0
154.0
155.0
156.0
157.0
158.0
159.0
160.0
161.0
162.0
163.0
164.0
165.0
166.0
167.0
168.0
169.0
170.0
171.0
172.0
173.0
174.0
175.0
176.0
177.0
178.0
179.0
180.0
181.0
182.0
183.0
184.0
185.

In [79]:
np_train_speed = np.array(train_speed)
np_train_topic = np.array(train_topic)
#np_history_speed = np.array(history_speed)
print(np_all_train.shape)
print(np_train_speed.shape,np_train_topic.shape)

(103660, 14, 8)
(103660, 10) (103660, 10, 8)


In [80]:
np.save(r'\np_train_speed_10s.npy',np_train_speed)
np.save(r'\np_train_topic_10s.npy',np_train_topic)
np.save(r'\np_all_train_10s.npy',np_all_train)
#np.save(r'D:\科研\SNN\new data\np_history_speed_10s.npy',np_history_speed)

In [85]:
np_history_speed = np.zeros([len(history_speed),12])
for i in range(len(history_speed)):
    for j in range(len(history_speed[i])):
        np_history_speed[i,2*j] = history_speed[i][j][0]
        np_history_speed[i,2*j+1] = history_speed[i][j][1]

In [86]:
np.save(r'\np_history_speed_10s.npy',np_history_speed)